In [ ]:
!apt install lshw
!curl -fsSL https://ollama.com/install.sh | sh
%pip install -q llama-index-llms-ollama

In [ ]:
%%capture
!git clone https://github.com/sabeeralikp/llama_flow.git
%cd llama_flow
!git switch "v0.0.2-alpha-remote-colab"
%pip install -q -r requirements.txt
%cd app

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
# from main import app
import multiprocessing
import asyncio

async def run(cmd):
  '''
  run is a helper function to run subcommands asynchronously.
  '''
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )


auth_token = "paste_your_ngrok_auth_token_here"

ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

await asyncio.gather(
    run(['ollama', 'serve']),
    run(['ollama', 'pull', 'phi3']),
    run(['uvicorn', 'main:app', '--port', '8000']),
)